In [19]:
import numpy as np
# read data from reviews and labels file.
with open('Datasets/all.csv', 'r') as f:
    all_accents = f.readlines() #len = 1679154

In [20]:
words, accents = [], []

In [21]:
for i in range(len(all_accents)):
    str = all_accents[i].replace('\n', '').split(',')
    words.append(str[0])
    accents.append(str[1])
max_len = len(max(words, key=len))

In [22]:
print('Максимальная длина слова:')
print(max_len)

Максимальная длина слова:
30


In [23]:
words_int, accents_int, bad_ex = [], [], []

In [24]:
for i in range(len(words)):
    if (len(words[i]) == len(accents[i])):
        bad_ex.append(words[i])
        #accents_int.append(accents[i].find(chr(39)))
    else:
        temp = []
        slovo = np.zeros(30)
        for j in range(len(words[i])):
            slovo[j]=ord(words[i][j])
        words_int.append(slovo)
        accents_int.append(accents[i].find(chr(39)))

In [25]:
for i in range(len(bad_ex)):
    print(bad_ex[i])

б
брр
в
взметывавший
впекшийся
гм
гнетшийся
гребаный
гребнутый
гребший
гребшийся
гуфсин
гэс
дакрис
дежка
дема
дернутый
десенный
джонни
дленный
дремный
ебывать
евич
ежащий
еженный
еживший
ежимый
ежистый
ейск
екавший
екарный
е-мое
епт
епть
ерзанье
ерзнуть
ерничанье
ерничество
ешкин
ж
жевка
жегшийся
жженный
звездность
зеванный
зленный
изя
итк
ифнс
ия
к
квелость
кельн
кельнский
керлер
кешк
кин
клеванный
клево
клевый
клеклый
клекнуть
клепанный
клец
ковая
ктор
кш
кънки
легочник
леживавший
леживающий
лежкий
лежкость
лель
леля
леня
лер
лесовый
летчицкий
лец
леч
ль
лькаля
мебиуса
межзвездный
мельн
мендово
мерзший
мерс
мертвыи
мерший
метанный
метнутый
метший
метшийся
мешреберный
мирович
мицгол
мицголе
мицголом
мм
мнев
мсье
мфу
мшенный
нди
нень
нио
нский
обь
омка
омск
орск
офсин
пекшийся
пернуть
пертый
петр
пехом
плев
пленочка
плесканный
плеснутый
плесо
плеточный
плетшийся
пнев
пневый
прдjети
псел
пшел
пшенник
ренн
рец
рэс
с
сас
сблевывавший
сблевывающий
сбреханный
сбрехнутый
свезшийся
сверлышко


In [43]:
train_frac = 0.8
val_frac = 0.1
test_frac = 0.1


def train_test_val_split(words_int):
    return (words_int[:int(len(words_int)*train_frac)], 
words_int[int(len(words_int)*train_frac) : (int(len(words_int)*train_frac) + int(len(words_int)*val_frac))], 
words_int[(int(len(words_int)*train_frac) + int(len(words_int)*val_frac)):])
    
def train_test_val_labels(accents_int):
    return (accents_int[:int(len(accents_int)*train_frac)], 
accents_int[int(len(accents_int)*train_frac) : (int(len(accents_int)*train_frac) + int(len(accents_int)*val_frac))], 
accents_int[(int(len(accents_int)*train_frac) + int(len(accents_int)*val_frac)):])

train_x, val_x, test_x = train_test_val_split(words_int)
train_y, val_y, test_y = train_test_val_labels(accents_int)

In [44]:
## print out the shapes of your resultant feature data
print("\t\t\t Sizes:")
print("Train set: \t\t{}".format(len(train_x)), 
      "\nValidation set: \t{}".format(len(val_x)),
      "\nTest set: \t\t{}".format(len(test_x)))

			 Sizes:
Train set: 		1342976 
Validation set: 	167872 
Test set: 		167872


In [47]:
print("\t\t Vizualization:")
train = ''
for i in range(10):
    train = ''
    for j in range(len(train_x[i])):
        train = train + chr(train_x[i][j])
    print("Train set: ",train)

for i in range(10):    
    vval = ''
    for j in range(len(val_x[i])):
        vval = vval + chr(val_x[i][j])
    print("Validation set: ",vval)

for i in range(10):
    test = ''
    for j in range(len(test_x[i])):
        test = test + chr(test_x[i][j])
    print("Test set: ",test)


		 Vizualization:
Train set:  а
Train set:  аав
Train set:  аава
Train set:  аавам
Train set:  аавами
Train set:  аавасакса
Train set:  аавах
Train set:  ааве
Train set:  аавик
Train set:  аавов
Validation set:  самообразовательный
Validation set:  самообразовательным
Validation set:  самообразовательными
Validation set:  самообразовательных
Validation set:  самообслуживавшийся
Validation set:  самообслуживавшись
Validation set:  самообслуживаемся
Validation set:  самообслуживаетесь
Validation set:  самообслуживается
Validation set:  самообслуживаешься
Test set:  трудоустройству
Test set:  трудоустройся
Test set:  трудоустроит
Test set:  трудоустроите
Test set:  трудоустройте
Test set:  трудоустроитесь
Test set:  трудоустройтесь
Test set:  трудоустроится
Test set:  трудоустроить
Test set:  трудоустроиться


In [49]:
print(type(train_y))

<class 'list'>


In [9]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# create Tensor datasets for train, test and val
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(val_x), torch.from_numpy(val_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))

# dataloaders
batch_size = 50 

# make sure to SHUFFLE your training data. Keep Shuffle=True.
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

TypeError: expected np.ndarray (got list)